In [1]:
# 테스트 세트를 사용해 자꾸 성능을 확인하다 보면 점점 테스트 세트에 맞춰지는 문제 발생
# 훈련세트, 검증 세트, 테스트 세트로 분리

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
# train_input과 train_target을 다시 훈련세트와 검증세트로 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [4]:
# 훈련세트와 검증세트 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
# 모델 생성
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련세트에 과대적합

0.9971133028626413
0.864423076923077


In [ ]:
# 3-폴드 교차 검증 -> 훈련세트를 세부분으로 나눠서 교차 검증 수헹(k-폴드 교차검증), 각 폴드에서 계산한 검증 점수를 평균함
# 평가할 모델 객체를 첫 번째 매개변수로 전달(직접 검증 세트를 떼어 내지X)
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores) # 첫 번째 키와 두 번째 키는 각각 훈련시간, 검증시간을 반환, 세 번째는 검증 폴드의 점수

{'fit_time': array([0.00597668, 0.00599909, 0.00711989, 0.00603843, 0.00596619]), 'score_time': array([0.00137401, 0.00154138, 0.00157976, 0.00130033, 0.00142837]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
# test_score키에 담긴 5개의 평균 점수 확인 -> 검증 폴드의 점수
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 교차검증을 할 때 훈련세트를 섞으려면 분할기를 사용하여야함
# 회귀모델 -> KFold 분할기, 분류모델 -> StratifiedKFold 사용

0.855300214703487


In [11]:
# 훈련세트를 섞고, 10-폴드 교차 검증
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
# 하이퍼파라미터 탐색과 교차검증을 동시에 수행하는 사이킷런의 GridSearchCV 클래스
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# fit()함수를 호출하여 파라매터 값을 바꿔가며 결정 트리 모델을 사용해 교차검증 5회 실행
# cv매개변수의 기본값은 5이므로 총 5*5=25개의 모델을 훈련
# 사용할 cpu 코어를 n_jobs 매개변수로 지정, -1의 경우 시스템에 있는 모든 코어 사용
gs.fit(train_input, train_target)

# 사이킷 런의 그리드 서치는 훈련이 끝나면 교차검증점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
# 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있음
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# 그리드서치로 찾은 최적의 매개변수 확인
print(gs.best_params_)

# 5번의 교차검증으로 얻은 점수 출력
print(gs.cv_results_['mean_test_score'])

{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [14]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [15]:
# 하이퍼 파라미터 튜닝 과정 정리
# 1. 탐색할 매개 변수 지정
# 2. 훈련세트에서 그리드 서치를 수행, 최적의 매개변수 조합 찾기
# 3. 그리드 섳치는 최상의 매개변수에서(교차검증에 사용된 훈련세트가 아닌) 전체 훈련세트를 사용해 최종모델을 훈련 및 그리드 서치 객체에 저장

In [17]:
# 더 복잡한 매개변수 조합 탐색
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)
          }
# 교차검증 횟수 : 9 * 15 * 10 = 1,350개 -> 5-폴드 교차검증 -> 6,750개

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

# 최상의 매개변수 조합 확인
print(gs.best_params_) 
# 최상의 교차검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}
0.8683865773302731


In [22]:
# 랜덤서치 -> 매개변수 값이 수치이고, 범위나 간격을 정하기 어려울 때
from scipy.stats import uniform, randint
# uniform, randint은 주어진 범위에서 고르게 값을 뽑는다.(균등분포에서 샘플링한다.) 각각 실숫값과 정숫값을 뽑는다. 
rgen = randint(0,10)
print(rgen.rvs(10))
ugen = uniform(0,1)
print(ugen.rvs(10))

[6 5 2 4 3 6 8 7 7 2]
[0.5551324  0.9742862  0.74326754 0.501217   0.70926787 0.50559377
 0.46224508 0.00176261 0.57642755 0.59086676]


In [ ]:
# 랜덤서치를 적용하여 하이퍼파라미터 탐색

# 1. 탐색할 매개변수 지정
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25) # 리프 노드가 되기 위한 최소 샘플 개수
          }

In [24]:
# 2. 최적의 매개변수 조합 찾기
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)
# 그리드 서치보다 교차검증 수를 훨씬 줄이면서 넓은 영역을 효과적으로 탐색 가능

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'max_depth': range(5, 20), 'min_impurity_decrease': array([0.0001... 0.0009]), 'min_samples_split': range(2, 100, 10)}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [25]:
# 3. 결과 확인
print(rs.best_params_)

{'min_samples_split': 12, 'min_impurity_decrease': np.float64(0.0005), 'max_depth': 11}


In [26]:
# 최고의 교차 검증 점수 확인
print(np.max(rs.cv_results_['mean_test_score']))

0.8681935292811135


In [27]:
# 전체 훈련세트로 훈련되어 저장된 모델을 최종 모델로 결정하고 테스트 세트의 성능확인
dt = rs.best_estimator_
print(dt.score(test_input, test_target)) 

0.8615384615384616
